In [ ]:
#!pip install --upgrade google-cloud-bigquery

In [ ]:
from google.cloud import bigquery
import pandas as pd
from google.colab import auth

# Google servislerine erişim sağlamak için kullanıcı kimliğini doğrulayan komuttur.
auth.authenticate_user()

# BigQuery istemcisini başlatma
def get_bigquery_client(project_id):
    return bigquery.Client(project=project_id)

# SQL sorgusunu çalıştırma fonksiyonu
def run_query(query, project_id):
    client = get_bigquery_client(project_id)
    query_job = client.query(query)
    return query_job.result().to_dataframe()

# BigQuery üzerinde çalışılacak projenin unique tanımlayıcısıdır. (project ID)
project_id = "mimic-4-ed"

In [ ]:
# Veri çerçevesi çıktılarının daha okunabilir olması için Pandas görüntüleme ayarları yapıldı.

pd.set_option("display.max_columns",None)
pd.set_option("display.width",600)
pd.set_option("display.max_rows",600)
pd.set_option("display.float_format",lambda x:"%.2f" %x)

 **Sepsis Tanılı ICU Hastalarının Filtrelenmesi**

Bu sorguda, literatüre uygun olarak sepsis tanısı alan ve anlamlı klinik verilere sahip hastalar filtrelenmiştir:



*  ICD-9 kodlarına göre sepsis tanılı hastalar alındı:
99591, 99592 ve 78552 kodları ile sepsis, ağır sepsis ve septik şok tanısı olan hastalar seçildi.

* SOFA skoru > 2 olanlar dahil edildi:
Sepsis-3 tanımına göre bu eşik değeri, organ disfonksiyonu göstergesi olarak kabul edilir.

* Yalnızca 18 yaşından büyük hastalar alındı:
Pediatrik hastalar dışlanarak sonuçların daha tutarlı olması sağlandı.

* ICU'da en az 24 saat kalanlar seçildi:
Genellikle klinik analizlerde ilk 24 saatlik veriler kullanıldığından bu kriter uygulandı.

* Sadece tek ICU yatışı olan hastalar alındı:
İlk ve tek ICU deneyimi olan hastalar analiz edildi, çünkü tekrarlayan yatışlar sonucu etkileyebilir.

In [ ]:
query = """
WITH diagnosis_data AS (
    SELECT
        diag.subject_id,
        diag.hadm_id
    FROM physionet-data.mimiciv_3_1_hosp.diagnoses_icd AS diag
    LEFT JOIN physionet-data.mimiciv_3_1_hosp.d_icd_diagnoses AS d_icd
        ON diag.icd_code = d_icd.icd_code
    WHERE diag.icd_version = 9
    AND diag.icd_code IN ('99591', '99592', '78552')  -- Sepsis, severe sepsis, septic shock
),

sepsis_data AS (
    SELECT
        subject_id,
        stay_id,
        sofa_score
    FROM mimic-4-ed.mimic4.sepsis3
    WHERE sofa_score > 2
),

icu_data AS (
    SELECT
        subject_id,
        stay_id,
        hadm_id,
        intime,
        outtime,
        los AS icu_los,
        first_careunit,
        last_careunit
    FROM physionet-data.mimiciv_3_1_icu.icustays
    WHERE los >= 1  -- En az 24 saat ICU'da kalanlar
),

age_data AS (
    SELECT subject_id, age
    FROM `mimic-4-ed.mimic4.age`
    WHERE age >= 18  -- 18 yaşından büyük hastalar
),

record_counts AS (
    SELECT
        subject_id,
        COUNT(DISTINCT stay_id) AS record_count
    FROM icu_data
    GROUP BY subject_id
),

ranked_data AS (
    SELECT
        icu.subject_id,
        icu.hadm_id,
        icu.stay_id,
        icu.intime,
        icu.outtime,
        FORMAT('%.2f', icu.icu_los) AS icu_los,
        icu.first_careunit,
        icu.last_careunit,
        s.sofa_score,
        age_data.age,
        record_counts.record_count,
        ROW_NUMBER() OVER (PARTITION BY icu.subject_id ORDER BY icu.intime DESC) AS rn
    FROM icu_data AS icu
    LEFT JOIN sepsis_data s
        ON icu.subject_id = s.subject_id AND icu.stay_id = s.stay_id
    LEFT JOIN age_data
        ON icu.subject_id = age_data.subject_id
    LEFT JOIN record_counts
        ON icu.subject_id = record_counts.subject_id
    INNER JOIN diagnosis_data d
        ON icu.hadm_id = d.hadm_id  -- ICD-9 tanı filtresi
    WHERE s.sofa_score IS NOT NULL
),

final_data AS (
    SELECT
        subject_id,
        hadm_id,
        stay_id,
        icu_los,
        age,
        sofa_score,
        record_count
    FROM ranked_data
    WHERE rn = 1 AND record_count = 1
)

SELECT * FROM final_data
ORDER BY sofa_score DESC;


"""

df = run_query(query, project_id)
df

,subject_id,hadm_id,stay_id,icu_los,age,sofa_score,record_count
0,16794039,22579763,30606871,8.39,60,20,1
1,19086192,25838193,36892955,1.99,80,16,1
2,18741255,23948220,38255896,36.00,61,15,1
3,15113669,26154533,31136730,1.66,49,15,1
4,17660889,26589042,35890155,15.77,51,14,1
...,...,...,...,...,...,...,...
1678,13921082,23709637,30324545,1.16,93,3,1
1679,14285895,20912504,32226811,2.86,30,3,1
1680,14362405,25527523,31270316,4.54,71,3,1
1681,14851496,20416083,37602163,30.91,44,3,1


**Deliryum ile İlgili Ölçüm Etiketlerinin Sorgulanması**

*d_items* tablosundan *cam-icu* ve *delirium* kelimelerini içeren etiketlere sahip benzersiz itemid ve label değerleri sorgulandı.

In [ ]:
query2 = """
SELECT DISTINCT itemid, label
FROM physionet-data.mimiciv_3_1_icu.d_items
WHERE LOWER(label) LIKE '%cam-icu%' OR LOWER(label) LIKE '%delirium%';
"""

df2 = run_query(query2, project_id)
df2

,itemid,label
0,228300,CAM-ICU MS change
1,228301,CAM-ICU Inattention
2,228302,CAM-ICU RASS LOC
3,228303,CAM-ICU Disorganized thinking
4,228332,Delirium assessment
5,228334,CAM-ICU Altered LOC
6,228335,CAM-ICU Disorganized thinking
7,228336,CAM-ICU Inattention
8,228337,CAM-ICU MS Change
9,229324,CAM-ICU Disorganized thinking


**Deliryum Tespiti (CAM-ICU, İlk 24 Saat)**

Bu sorguda, ICU hastalarında ilk 24 saatte CAM-ICU kriterlerine göre deliryum tanısı değerlendirilmiştir. Kriterler:

* Mental durum değişikliği
* Dikkat eksikliği
* Düşünce bozukluğu veya bilinç bozukluğu

Yatıştan sonraki 24 saat içinde ölçüm değeri “1” veya “yes” olan kayıtlar alınmıştır. Deliryum pozitif sayılmak için üç kriterin sağlanması gerekir.

**Deliryumu Etkileyebilecek Diğer Durumların Hariç Tutulması**

CAM-ICU pozitifliğinin gerçek deliryumu yansıtması için, bazı hastalar analiz dışı bırakılmıştır. Hariç tutulanlar:

* Demans, psikotik bozukluklar
* Alkol/madde kullanımı
* Epilepsi, nöbet, kafa travması
* Elektrolit dengesizlikleri
* Zehirlenme, ilaç toksisitesi

Bu hastalıklara ait icd kodları bigquery üzerinde tespit edildi ve ona göre filtreleme yapıldı.

**Komorbiditeler ve Klinik Göstergeler**

Veri setine dahil edilen bazı sağlık sorunları:

* Diyabet
* Hipertansiyon
* Böbrek/Karaciğer/Kalp hastalıkları

Bu komorbiditelere ait icd kodları bigquery üzerinde tespit edildi ve ona göre filtreleme yapıldı.

**Ayrıca klinik skorlar da dahil edilmiştir:**

Bu klinik skorlar, MIMIC GitHub hesabında yer alan sorgulardan oluşturulan tablolardan alınmıştır
* SOFA
* GCS
* SAPS II

In [ ]:
query6 = """CREATE TABLE `mimic-4-ed.mimic4.final_son4` AS (
WITH diagnosis_data AS (
    SELECT diag.subject_id, diag.hadm_id
    FROM physionet-data.mimiciv_3_1_hosp.diagnoses_icd AS diag
    LEFT JOIN physionet-data.mimiciv_3_1_hosp.d_icd_diagnoses AS d_icd
        ON diag.icd_code = d_icd.icd_code
    WHERE diag.icd_version = 9
      AND diag.icd_code IN ('99591', '99592', '78552')
),

sepsis_data AS (
    SELECT subject_id, stay_id, sofa_score
    FROM mimic-4-ed.mimic4.sepsis3
    WHERE sofa_score > 2
),

comorbidities AS (
  SELECT
    d.subject_id,
    d.hadm_id,
    MAX(CASE WHEN d.icd_code LIKE '250%' THEN 1 ELSE 0 END) AS diabetes,
    MAX(CASE WHEN d.icd_code LIKE '401%' THEN 1 ELSE 0 END) AS hypertension,
    MAX(CASE WHEN d.icd_code LIKE '584%' THEN 1 ELSE 0 END) AS renal_failure,
    MAX(CASE WHEN d.icd_code LIKE '490%' THEN 1 ELSE 0 END) AS pulmonary,
    MAX(CASE WHEN d.icd_code BETWEEN '390' AND '459' THEN 1 ELSE 0 END) AS cardiovascular,
    MAX(CASE WHEN d.icd_code LIKE '570%' THEN 1 ELSE 0 END) AS liver_failure,
    MAX(CASE WHEN d.icd_code LIKE '410%' THEN 1 ELSE 0 END) AS ami,
    MAX(CASE WHEN d.icd_code LIKE '585%' THEN 1 ELSE 0 END) AS ckd,
    MAX(CASE WHEN d.icd_code BETWEEN '490' AND '496' THEN 1 ELSE 0 END) AS copd,
    MAX(CASE WHEN d.icd_code LIKE '428%' THEN 1 ELSE 0 END) AS heart_failure,
    MAX(CASE WHEN d.icd_code BETWEEN '140' AND '209' THEN 1 ELSE 0 END) AS malignant_tumor
  FROM physionet-data.mimiciv_3_1_hosp.diagnoses_icd d
  WHERE d.icd_version = 9
  GROUP BY d.subject_id, d.hadm_id
),

icu_data AS (
    SELECT subject_id, stay_id, hadm_id, intime, outtime,
           los AS icu_los, first_careunit, last_careunit
    FROM physionet-data.mimiciv_3_1_icu.icustays
    WHERE los >= 1
),

age_data AS (
    SELECT subject_id, age
    FROM `mimic-4-ed.mimic4.age`
    WHERE age >= 18
),

sex_data AS (
    SELECT subject_id, gender AS sex
    FROM physionet-data.mimiciv_3_1_hosp.patients
),

record_counts AS (
    SELECT subject_id, COUNT(DISTINCT stay_id) AS record_count
    FROM icu_data
    GROUP BY subject_id
),

gcs_data AS (
    SELECT subject_id, stay_id, AVG(gcs) AS gcs
    FROM `mimic-4-ed.mimic4.gcs`
    GROUP BY subject_id, stay_id
),

sapsii_data AS (
    SELECT subject_id, stay_id, sapsii
    FROM `mimic-4-ed.mimic4.sapsii`
),

charlson_data AS (
    SELECT subject_id, hadm_id, charlson_comorbidity_index
    FROM mimic-4-ed.mimic4.charlson
),

-- CAM-ICU adımı (sadece ilk 24 saat)
cam_icu_raw AS (
  SELECT ce.subject_id, ce.stay_id, ce.charttime, ce.itemid, ce.valuenum,
         CASE
           WHEN ce.itemid IN (228300, 228337, 229326) THEN 'ms_change'
           WHEN ce.itemid IN (228301, 228336, 229325) THEN 'inattention'
           WHEN ce.itemid IN (228303, 228335, 229324) THEN 'disorganized_thinking'
           WHEN ce.itemid IN (228302, 228334) THEN 'altered_loc'
         END AS feature
  FROM physionet-data.mimiciv_3_1_icu.chartevents AS ce
  JOIN physionet-data.mimiciv_3_1_icu.icustays AS icu
    ON ce.subject_id = icu.subject_id AND ce.stay_id = icu.stay_id
  WHERE ce.itemid IN (
    228300, 228301, 228302, 228303,
    228334, 228335, 228336, 228337,
    229324, 229325, 229326
  )
  AND (ce.valuenum = 1 OR LOWER(ce.value) = 'yes')
  AND DATETIME_DIFF(ce.charttime, icu.intime, HOUR) <= 24
),

cam_icu_grouped AS (
  SELECT subject_id, stay_id,
         MAX(IF(feature = 'ms_change', 1, 0)) AS ms_change,
         MAX(IF(feature = 'inattention', 1, 0)) AS inattention,
         MAX(IF(feature = 'disorganized_thinking', 1, 0)) AS disorg_thinking,
         MAX(IF(feature = 'altered_loc', 1, 0)) AS altered_loc
  FROM cam_icu_raw
  GROUP BY subject_id, stay_id
),

cam_icu_positive AS (
  SELECT *
  FROM cam_icu_grouped
  WHERE ms_change = 1 AND inattention = 1 AND (disorg_thinking = 1 OR altered_loc = 1)
),

excluded_delirium_causes AS (
  SELECT DISTINCT subject_id
  FROM physionet-data.mimiciv_3_1_hosp.diagnoses_icd
  WHERE icd_version = 9
    AND (
      icd_code LIKE '290%' OR icd_code LIKE '291%' OR icd_code LIKE '292%' OR icd_code LIKE '293%' OR
      icd_code LIKE '295%' OR icd_code LIKE '296%' OR icd_code LIKE '297%' OR icd_code LIKE '298%' OR
      icd_code LIKE '299%' OR icd_code LIKE '303%' OR icd_code LIKE '304%' OR icd_code LIKE '305%' OR
      icd_code LIKE '308%' OR icd_code LIKE '309%' OR icd_code LIKE '345%' OR icd_code = '3483' OR
      icd_code IN ('2761', '2768', '79902') OR
      icd_code BETWEEN '85000' AND '85499' OR
      icd_code = '95901' OR icd_code BETWEEN '96500' AND '98999'
    )
),

ranked_data AS (
    SELECT
        icu.subject_id,
        icu.hadm_id,
        icu.stay_id,
        icu.intime,
        icu.outtime,
        FORMAT('%.2f', icu.icu_los) AS icu_los,
        icu.first_careunit,
        icu.last_careunit,
        s.sofa_score,
        age_data.age,
        sex_data.sex,
        record_counts.record_count,
        FORMAT('%.2f', gcs_data.gcs) AS gcs,
        sapsii_data.sapsii,
        charlson_data.charlson_comorbidity_index,
        IF(cam.subject_id IS NOT NULL, 1, 0) AS delirium_positive,
        comorbidities.diabetes,
        comorbidities.hypertension,
        comorbidities.renal_failure,
        comorbidities.pulmonary,
        comorbidities.cardiovascular,
        comorbidities.liver_failure,
        comorbidities.ami,
        comorbidities.ckd,
        comorbidities.copd,
        comorbidities.heart_failure,
        comorbidities.malignant_tumor,
        ROW_NUMBER() OVER (PARTITION BY icu.subject_id ORDER BY icu.intime DESC) AS rn
    FROM icu_data AS icu
    LEFT JOIN sepsis_data s
        ON icu.subject_id = s.subject_id AND icu.stay_id = s.stay_id
    LEFT JOIN age_data
        ON icu.subject_id = age_data.subject_id
    LEFT JOIN sex_data
        ON icu.subject_id = sex_data.subject_id
    LEFT JOIN record_counts
        ON icu.subject_id = record_counts.subject_id
    LEFT JOIN gcs_data
        ON icu.subject_id = gcs_data.subject_id AND icu.stay_id = gcs_data.stay_id
    LEFT JOIN sapsii_data
        ON icu.subject_id = sapsii_data.subject_id AND icu.stay_id = sapsii_data.stay_id
    LEFT JOIN charlson_data
        ON icu.subject_id = charlson_data.subject_id AND icu.hadm_id = charlson_data.hadm_id
    LEFT JOIN comorbidities
        ON icu.subject_id = comorbidities.subject_id AND icu.hadm_id = comorbidities.hadm_id
    INNER JOIN diagnosis_data d
        ON icu.hadm_id = d.hadm_id
    LEFT JOIN cam_icu_positive cam
        ON icu.subject_id = cam.subject_id AND icu.stay_id = cam.stay_id
    LEFT JOIN excluded_delirium_causes excl
        ON icu.subject_id = excl.subject_id
    WHERE s.sofa_score IS NOT NULL AND excl.subject_id IS NULL
),

final_data AS (
    SELECT
        subject_id,
        hadm_id,
        stay_id,
        icu_los,
        age,
        sex,
        sofa_score,
        gcs,
        sapsii,
        charlson_comorbidity_index,
        record_count,
        delirium_positive,
        diabetes,
        hypertension,
        renal_failure,
        pulmonary,
        cardiovascular,
        liver_failure,
        ami,
        ckd,
        copd,
        heart_failure,
        malignant_tumor
    FROM ranked_data
    WHERE rn = 1 AND record_count = 1
)

SELECT * FROM final_data
ORDER BY sofa_score DESC);


"""

df6 = run_query(query6, project_id)
df6

""


**Mortality Etiketi Oluşturma**

Hastaneye yatıştan sonraki **14 gün içinde ölüm** gerçekleşip gerçekleşmediğine göre `mortality` etiketi oluşturulmuştur:

- **mortality = 1** → Hasta 14 gün içinde vefat etti  
- **mortality = 0** → Hasta 14 gün içinde vefat etmedi veya hayatta

Veri: `deathtime`, `admittime` → `DATE_DIFF` ile fark hesaplanmıştır.


In [ ]:
query10 = """
SELECT
  f.*,
  a.deathtime,
  DATE_DIFF(a.deathtime, a.admittime, DAY) AS death_day,
  CASE
    WHEN a.deathtime IS NOT NULL AND DATE_DIFF(a.deathtime, a.admittime, DAY) <= 14 THEN 1
    ELSE 0
  END AS mortality
FROM
  mimic-4-ed.mimic4.final_son4 f
LEFT JOIN
  physionet-data.mimiciv_3_1_hosp.admissions a
ON
  f.subject_id = a.subject_id
  AND f.hadm_id = a.hadm_id
"""

df10 = run_query(query10, project_id)
df10

,subject_id,hadm_id,stay_id,icu_los,age,sex,sofa_score,gcs,sapsii,charlson_comorbidity_index,record_count,delirium_positive,diabetes,hypertension,renal_failure,pulmonary,cardiovascular,liver_failure,ami,ckd,copd,heart_failure,malignant_tumor,deathtime,death_day,mortality
0,19086192,25838193,36892955,1.99,80,M,16,14.18,76,7,1,0,0,0,1,0,1,0,0,0,0,0,0,2135-10-09 12:45:00,2,1
1,18519121,24385819,38768547,5.45,58,F,14,14.55,71,2,1,0,0,0,0,0,0,0,0,0,0,0,0,NaT,<NA>,0
2,15371701,26268759,32666850,5.79,87,F,13,15.00,60,8,1,0,1,0,1,0,1,0,0,0,1,1,0,2176-02-14 11:30:00,6,1
3,15648679,26296326,36442642,3.09,64,F,13,15.00,67,5,1,0,0,0,1,0,1,1,1,0,0,0,0,2126-11-20 19:20:00,3,1
4,15274195,27623566,37800255,31.12,44,F,13,13.91,59,6,1,0,0,0,1,0,1,1,0,1,0,0,0,NaT,<NA>,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,15387306,24617002,34558292,1.17,71,M,3,12.75,36,7,1,0,0,0,1,0,1,0,0,1,0,1,0,NaT,<NA>,0
654,18022445,28407911,36390922,3.31,78,F,3,14.05,43,11,1,0,1,1,0,0,1,0,0,0,0,1,0,NaT,<NA>,0
655,17150377,24304553,34613772,2.00,63,M,3,13.00,50,13,1,0,1,0,1,0,0,0,0,0,0,0,1,2110-04-13 17:05:00,19,0
656,11006621,27253628,35041425,1.41,75,M,3,15.00,47,11,1,0,0,0,1,0,1,0,0,1,1,1,0,NaT,<NA>,0


Üst tarafta yer alan sorgunun tablo olarak kaydedildiği hali aşağıda yer almaktadır.

In [ ]:
query11 = """
CREATE TABLE `mimic-4-ed.mimic4.final_son5` AS (
  SELECT
    f.*,
    a.deathtime,
    DATE_DIFF(a.deathtime, a.admittime, DAY) AS death_day,
    CASE
      WHEN a.deathtime IS NOT NULL AND DATE_DIFF(a.deathtime, a.admittime, DAY) <= 14 THEN 1
      ELSE 0
    END AS mortality
  FROM
    `mimic-4-ed.mimic4.final_son4` f
  LEFT JOIN
    `physionet-data.mimiciv_3_1_hosp.admissions` a
  ON
    f.subject_id = a.subject_id
    AND f.hadm_id = a.hadm_id
);
"""

df11 = run_query(query11, project_id)


**Vital Bulguların Tanımlanması**

Bu adımda, ICU'da izlenen bazı hayati parametrelerin açıklamaları `d_items` tablosundan çekilmiştir.

- `itemid`, `label`, `category` bilgileri getirilmiştir.
- Seçilen `itemid` değerleri, projede kullanılacak vital bulgulara karşılık gelir (ör. kalp hızı, kan basıncı, solunum vb.).
- Sadece belirtilen 10 `itemid` filtrelenmiştir.


In [ ]:
query7 = """
SELECT
  itemid,
  label,
  category
FROM
  physionet-data.mimiciv_3_1_icu.d_items
WHERE
  itemid IN (
    220052, 220224, 220235, 225668, 220045, 220050, 220051, 220210,
    220277,227466
  )
ORDER BY itemid;
"""

df7 = run_query(query7, project_id)
df7

,itemid,label,category
0,220045,Heart Rate,Routine Vital Signs
1,220050,Arterial Blood Pressure systolic,Routine Vital Signs
2,220051,Arterial Blood Pressure diastolic,Routine Vital Signs
3,220052,Arterial Blood Pressure mean,Routine Vital Signs
4,220210,Respiratory Rate,Respiratory
5,220224,Arterial O2 pressure,Labs
6,220235,Arterial CO2 Pressure,Labs
7,220277,O2 saturation pulseoxymetry,Respiratory
8,225668,Lactic Acid,Labs
9,227466,PTT,Labs


**Laboratuvar Testlerinin Tanımlanması**

Bu adımda, analizde kullanılacak önemli laboratuvar testlerinin tanımları `d_labitems` tablosundan alınmıştır.


In [ ]:
query8 = """
SELECT
  itemid,
  label,
  fluid,
  category
FROM
  physionet-data.mimiciv_3_1_hosp.d_labitems
WHERE
  itemid IN (
    50861, 50863, 50960, 50811, 50893, 50970, 50825, 50882, 50868, 50820,
    50902, 50912, 50931, 50983, 50971, 51265, 51516, 50862, 51256, 50817,
    50810, 50813, 50878, 50885, 51274, 51237, 51006
  )
ORDER BY itemid;
"""

df8 = run_query(query8, project_id)
df8

,itemid,label,fluid,category
0,50810,"Hematocrit, Calculated",Blood,Blood Gas
1,50811,Hemoglobin,Blood,Blood Gas
2,50813,Lactate,Blood,Blood Gas
3,50817,Oxygen Saturation,Blood,Blood Gas
4,50820,pH,Blood,Blood Gas
5,50825,Temperature,Blood,Blood Gas
6,50861,Alanine Aminotransferase (ALT),Blood,Chemistry
7,50862,Albumin,Blood,Chemistry
8,50863,Alkaline Phosphatase,Blood,Chemistry
9,50868,Anion Gap,Blood,Chemistry


**İlk 24 Saatteki Vital Bulgular ve Laboratuvar Verileri**

Bu sorgu ile, ICU'ya kabul edilen hastaların ilk 24 saat içinde yapılan vital bulgu ölçümleri ve laboratuvar test sonuçları, hasta bilgileri ve etiketlerle birleştirilmiştir. Amaç, her hastanın belirli zaman dilimlerinde yapılan ölçüm değerlerini detaylı şekilde görüntülemektir.



In [ ]:

query9 = """
-- CHARTEVENTS
SELECT
  ce.subject_id,
  ce.stay_id,
  cohort.icu_los,
  cohort.age,
  cohort.sofa_score,
  cohort.gcs,
  cohort.sapsii,
  cohort.charlson_comorbidity_index,
  cohort.record_count,
  cohort.delirium_positive,
  cohort.mortality,
  ce.charttime AS charttime,
  icu.intime AS intime,
  ce.itemid,
  ce.valuenum,
  cohort.diabetes,
  cohort.hypertension,
  cohort.renal_failure,
  cohort.pulmonary,
  cohort.cardiovascular,
  cohort.liver_failure,
  cohort.ami,
  cohort.ckd,
  cohort.copd,
  cohort.heart_failure,
  cohort.malignant_tumor
FROM
  physionet-data.mimiciv_3_1_icu.chartevents ce
JOIN
  mimic-4-ed.mimic4.final_son5 cohort
ON
  ce.subject_id = cohort.subject_id
  AND ce.stay_id = cohort.stay_id
JOIN
  physionet-data.mimiciv_3_1_icu.icustays icu
ON
  ce.stay_id = icu.stay_id
WHERE
  ce.itemid IN (220052, 220546, 220228, 225668, 220045, 220179, 220180, 220210, 220277, 227466)
  AND DATETIME_DIFF(ce.charttime, icu.intime, HOUR) BETWEEN 0 AND 24
  AND ce.valuenum IS NOT NULL

UNION ALL

-- LABEVENTS (itemid 5 ile başlayanlar)
SELECT
  le.subject_id,
  cohort.stay_id,
  cohort.icu_los,
  cohort.age,
  cohort.sofa_score,
  cohort.gcs,
  cohort.sapsii,
  cohort.charlson_comorbidity_index,
  cohort.record_count,
  cohort.delirium_positive,
  cohort.mortality,
  le.charttime AS charttime,
  icu.intime AS intime,
  le.itemid,
  le.valuenum,
  cohort.diabetes,
  cohort.hypertension,
  cohort.renal_failure,
  cohort.pulmonary,
  cohort.cardiovascular,
  cohort.liver_failure,
  cohort.ami,
  cohort.ckd,
  cohort.copd,
  cohort.heart_failure,
  cohort.malignant_tumor
FROM
  physionet-data.mimiciv_3_1_hosp.labevents le
JOIN
  mimic-4-ed.mimic4.final_son5 cohort
ON
  le.subject_id = cohort.subject_id
JOIN
  physionet-data.mimiciv_3_1_icu.icustays icu
ON
  cohort.stay_id = icu.stay_id
WHERE
  le.itemid IN (
    50861, 50863, 50960, 50893, 50970, 50825, 50882, 50868, 50820,
    50902, 50912, 50931, 50983, 50971, 51265, 51256, 50817,
    50813, 50878, 50885, 51274, 51237, 51006, 50862, 50818, 50821,52170
  )
  AND DATETIME_DIFF(le.charttime, icu.intime, HOUR) BETWEEN 0 AND 24
  AND le.valuenum IS NOT NULL
ORDER BY
  subject_id,
  charttime;
"""

df9 = run_query(query9, project_id)
df9

,subject_id,stay_id,icu_los,age,sofa_score,gcs,sapsii,charlson_comorbidity_index,record_count,delirium_positive,mortality,charttime,intime,itemid,valuenum,diabetes,hypertension,renal_failure,pulmonary,cardiovascular,liver_failure,ami,ckd,copd,heart_failure,malignant_tumor
0,10017851,39558308,1.25,75,4,15.00,60,6,1,0,1,2115-11-21 20:03:00,2115-11-21 19:49:00,220179,109.00,0,0,1,0,1,0,0,0,1,1,0
1,10017851,39558308,1.25,75,4,15.00,60,6,1,0,1,2115-11-21 20:03:00,2115-11-21 19:49:00,220180,76.00,0,0,1,0,1,0,0,0,1,1,0
2,10017851,39558308,1.25,75,4,15.00,60,6,1,0,1,2115-11-21 20:04:00,2115-11-21 19:49:00,220045,134.00,0,0,1,0,1,0,0,0,1,1,0
3,10017851,39558308,1.25,75,4,15.00,60,6,1,0,1,2115-11-21 20:04:00,2115-11-21 19:49:00,220210,20.00,0,0,1,0,1,0,0,0,1,1,0
4,10017851,39558308,1.25,75,4,15.00,60,6,1,0,1,2115-11-21 20:14:00,2115-11-21 19:49:00,220277,76.00,0,0,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135994,19974380,35371457,3.46,78,6,12.89,68,7,1,0,0,2182-07-02 09:04:00,2182-07-01 09:33:18,220180,78.00,0,0,1,0,1,0,0,1,0,0,0
135995,19974380,35371457,3.46,78,6,12.89,68,7,1,0,0,2182-07-02 09:04:00,2182-07-01 09:33:18,220179,156.00,0,0,1,0,1,0,0,1,0,0,0
135996,19974380,35371457,3.46,78,6,12.89,68,7,1,0,0,2182-07-02 09:25:00,2182-07-01 09:33:18,51274,14.90,0,0,1,0,1,0,0,1,0,0,0
135997,19974380,35371457,3.46,78,6,12.89,68,7,1,0,0,2182-07-02 09:25:00,2182-07-01 09:33:18,51237,1.40,0,0,1,0,1,0,0,1,0,0,0


**Hasta başına, her gözlem türü için aşağıdaki istatistiksel özetler hesaplanmıştır:**


* Minimum (min)
* Medyan (median)
* Maksimum (max)
* Ortalama (avg)


Hesaplanan bu öznitelikler, itemid'ye karşılık gelen anlaşılır değişken adlarıyla yeniden adlandırılmıştır (örneğin: Creatinine_avg, HeartRate_max).


In [ ]:
itemid_name_map = {
    50861: "ALT",
    50863: "AlkalinePhosphatase",
    50960: "Magnesium",
    220228: "Hemoglobin",
    50893: "TotalCalcium",
    50970: "Phosphate",
    50825: "Temperature",
    50882: "Bicarbonate",
    50868: "AnionGap",
    50820: "PH",
    50902: "Chloride",
    50912: "Creatinine",
    50931: "Glucose",
    50983: "Sodium",
    50971: "Potassium",
    51265: "Platelets",
    220546: "WBC",
    50862: "Albumin",
    51256: "Neutrophils",
    50817: "OxygenSaturation",
    226540: "Hematocrit",
    50813: "LactateDehydrogenase",
    50878: "AST",
    52170: "RBG",
    50885: "TotalBilirubin",
    51274: "ProthrombinTime_PT",
    51237: "INR",
    51006: "BUN",
    227466: "aPTT",
    220052: "MeanArterialPressure",
    50821: "PaO2",
    50818: "PCO2",
    225668: "LacticAcid",
    220045: "HeartRate",
    220179: "SystolicBP",
    220180: "DiastolicBP",
    220210: "SedationScore",
    220277: "SpO2"
}


def pivot_item_statistics_with_names(df, itemid_name_map):
    df = df[df['itemid'].isin(itemid_name_map.keys())]
    df = df[['subject_id', 'itemid', 'valuenum']]


    stats = df.groupby(['subject_id', 'itemid'])['valuenum'].agg(
        min_value='min',
        median_value=lambda x: x.median(),
        max_value='max',
        avg_value='mean'
    ).reset_index()

    stats_long = pd.melt(stats, id_vars=['subject_id', 'itemid'], var_name='stat', value_name='value')

    stats_long['feature_name'] = stats_long.apply(
        lambda row: f"{itemid_name_map[row['itemid']]}_{row['stat'].replace('_value', '')}", axis=1
    )

    stats_pivot = stats_long.pivot(index='subject_id', columns='feature_name', values='value')


    ordered_cols = []
    for itemid, name in itemid_name_map.items():
        for stat in ['min', 'median', 'max', 'avg']:
            col = f"{name}_{stat}"
            if col in stats_pivot.columns:
                ordered_cols.append(col)

    stats_pivot = stats_pivot[ordered_cols]
    stats_pivot.reset_index(inplace=True)

    return stats_pivot

In [ ]:
data = pivot_item_statistics_with_names(df9, itemid_name_map)
data

feature_name,subject_id,ALT_min,ALT_median,ALT_max,ALT_avg,AlkalinePhosphatase_min,AlkalinePhosphatase_median,AlkalinePhosphatase_max,AlkalinePhosphatase_avg,Magnesium_min,Magnesium_median,Magnesium_max,Magnesium_avg,Hemoglobin_min,Hemoglobin_median,Hemoglobin_max,Hemoglobin_avg,TotalCalcium_min,TotalCalcium_median,TotalCalcium_max,TotalCalcium_avg,Phosphate_min,Phosphate_median,Phosphate_max,Phosphate_avg,Temperature_min,Temperature_median,Temperature_max,Temperature_avg,Bicarbonate_min,Bicarbonate_median,Bicarbonate_max,Bicarbonate_avg,AnionGap_min,AnionGap_median,AnionGap_max,AnionGap_avg,PH_min,PH_median,PH_max,PH_avg,Chloride_min,Chloride_median,Chloride_max,Chloride_avg,Creatinine_min,Creatinine_median,Creatinine_max,Creatinine_avg,Glucose_min,Glucose_median,Glucose_max,Glucose_avg,Sodium_min,Sodium_median,Sodium_max,Sodium_avg,Potassium_min,Potassium_median,Potassium_max,Potassium_avg,Platelets_min,Platelets_median,Platelets_max,Platelets_avg,WBC_min,WBC_median,WBC_max,WBC_avg,Albumin_min,Albumin_median,Albumin_max,Albumin_avg,Neutrophils_min,Neutrophils_median,Neutrophils_max,Neutrophils_avg,OxygenSaturation_min,OxygenSaturation_median,OxygenSaturation_max,OxygenSaturation_avg,LactateDehydrogenase_min,LactateDehydrogenase_median,LactateDehydrogenase_max,LactateDehydrogenase_avg,AST_min,AST_median,AST_max,AST_avg,TotalBilirubin_min,TotalBilirubin_median,TotalBilirubin_max,TotalBilirubin_avg,ProthrombinTime_PT_min,ProthrombinTime_PT_median,ProthrombinTime_PT_max,ProthrombinTime_PT_avg,INR_min,INR_median,INR_max,INR_avg,BUN_min,BUN_median,BUN_max,BUN_avg,aPTT_min,aPTT_median,aPTT_max,aPTT_avg,MeanArterialPressure_min,MeanArterialPressure_median,MeanArterialPressure_max,MeanArterialPressure_avg,PaO2_min,PaO2_median,PaO2_max,PaO2_avg,PCO2_min,PCO2_median,PCO2_max,PCO2_avg,LacticAcid_min,LacticAcid_median,LacticAcid_max,LacticAcid_avg,HeartRate_min,HeartRate_median,HeartRate_max,HeartRate_avg,SystolicBP_min,SystolicBP_median,SystolicBP_max,SystolicBP_avg,DiastolicBP_min,DiastolicBP_median,DiastolicBP_max,DiastolicBP_avg,SedationScore_min,SedationScore_median,SedationScore_max,SedationScore_avg,SpO2_min,SpO2_median,SpO2_max,SpO2_avg
0,10017851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.50,1.50,1.50,1.50,10.60,10.60,10.60,10.60,7.30,7.30,7.30,7.30,4.60,4.60,4.60,4.60,33.70,35.30,36.90,35.30,22.00,22.00,22.00,22.00,18.00,18.00,18.00,18.00,7.14,7.25,7.26,7.22,105.00,105.00,105.00,105.00,1.70,1.70,1.70,1.70,127.00,127.00,127.00,127.00,140.00,140.00,140.00,140.00,4.50,4.50,4.50,4.50,492.00,492.00,492.00,492.00,20.40,20.40,20.40,20.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.00,63.00,98.00,63.00,3.40,4.10,4.60,4.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.40,50.40,50.40,50.40,5.50,5.50,5.50,5.50,52.00,52.00,52.00,52.00,42.70,42.70,42.70,42.70,-12.00,-11.00,-10.00,-11.00,18.00,45.00,181.00,81.33,51.00,54.00,72.00,59.00,3.40,4.10,4.60,4.03,105.00,126.00,148.00,126.58,80.00,101.00,135.00,100.15,38.00,56.00,85.00,58.39,8.00,24.00,35.00,22.82,76.00,96.00,100.00,93.70
1,10020944,15.00,23.50,32.00,23.50,105.00,109.00,113.00,109.00,1.30,1.60,1.80,1.57,10.00,10.60,11.20,10.60,7.60,8.20,8.30,8.03,1.20,1.50,4.80,2.50,NaN,NaN,NaN,NaN,24.00,25.00,27.00,25.33,8.00,8.50,9.00,8.50,7.13,7.28,7.44,7.28,103.00,110.00,111.00,108.00,1.20,1.20,1.40,1.27,34.00,64.00,124.00,74.00,132.00,140.00,141.00,137.67,4.30,4.55,4.80,4.55,151.00,158.50,166.00,158.50,9.30,11.00,12.70,11.00,3.30,3.30,3.30,3.30,76.20,79.10,82.00,79.10,NaN,NaN,NaN,NaN,1.10,2.10,3.10,2.10,21.00,69.50,118.00,69.50,0.40,0.40,0.40,0.40,12.60,13.05,13.50,13.05,1.20,1.20,1.20,1.20,31.00,31.00,34.00,32.00,32.60,33.30,34.00,33.30,57.00,77.00,113.00,79.28,30.00,80.00,101.00,76.00,38.00,53.00,84.00,56.00,1.10,2.10,3.10,2.10,53.00,56.00,78.00,58.31,101.00,113.00,113.00,109.00,59.00,62.00,69.00,63.33,6.00,16.00,28.00,17.32,93.00,97.00,100.00,96.83
2,10022584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.80,1.90,2.00,1.90,9.00,9.00,9.00,9.00,7.60,7.60,7.60,7.60,2.70,3.10,3.50,3.10,NaN,NaN,NaN,NaN,15.00,19.00,20.00,18.00,12.00,13.00,

In [ ]:
data.columns.to_list()

['subject_id',
 'ALT_min',
 'ALT_median',
 'ALT_max',
 'ALT_avg',
 'AlkalinePhosphatase_min',
 'AlkalinePhosphatase_median',
 'AlkalinePhosphatase_max',
 'AlkalinePhosphatase_avg',
 'Magnesium_min',
 'Magnesium_median',
 'Magnesium_max',
 'Magnesium_avg',
 'Hemoglobin_min',
 'Hemoglobin_median',
 'Hemoglobin_max',
 'Hemoglobin_avg',
 'TotalCalcium_min',
 'TotalCalcium_median',
 'TotalCalcium_max',
 'TotalCalcium_avg',
 'Phosphate_min',
 'Phosphate_median',
 'Phosphate_max',
 'Phosphate_avg',
 'Temperature_min',
 'Temperature_median',
 'Temperature_max',
 'Temperature_avg',
 'Bicarbonate_min',
 'Bicarbonate_median',
 'Bicarbonate_max',
 'Bicarbonate_avg',
 'AnionGap_min',
 'AnionGap_median',
 'AnionGap_max',
 'AnionGap_avg',
 'PH_min',
 'PH_median',
 'PH_max',
 'PH_avg',
 'Chloride_min',
 'Chloride_median',
 'Chloride_max',
 'Chloride_avg',
 'Creatinine_min',
 'Creatinine_median',
 'Creatinine_max',
 'Creatinine_avg',
 'Glucose_min',
 'Glucose_median',
 'Glucose_max',
 'Glucose_avg',
 '

In [ ]:
missing_counts = data.isnull().sum()

print(missing_counts.sort_values(ascending=False))

feature_name
OxygenSaturation_min           336
OxygenSaturation_avg           336
OxygenSaturation_max           336
OxygenSaturation_median        336
Albumin_avg                    328
Albumin_min                    328
Albumin_median                 328
Albumin_max                    328
MeanArterialPressure_min       328
MeanArterialPressure_avg       328
MeanArterialPressure_max       328
MeanArterialPressure_median    328
Temperature_avg                326
Temperature_min                326
Temperature_median             326
Temperature_max                326
Neutrophils_avg                263
Neutrophils_median             263
Neutrophils_max                263
Neutrophils_min                263
AlkalinePhosphatase_avg        191
AlkalinePhosphatase_max        191
AlkalinePhosphatase_min        191
AlkalinePhosphatase_median     191
AST_min                        189
ALT_avg                        189
ALT_min                        189
ALT_max                        189
ALT_med

**Veri setindeki eksik veriler analiz edilmiştir.**

İlk olarak, her satırdaki eksik veri oranı hesaplanmış ve %30’dan fazla eksik değere sahip satırlar (yani hastalar) veri setinden çıkarılmıştır. Bu işlem sonucunda 73 hasta veri setinden çıkarılmıştır.


In [ ]:
missing_ratio = data.isnull().mean(axis=1)

# %30'dan fazla eksik değeri olan satırları seç
filtered_df = data[missing_ratio > 0.3]
print("Kişi sayısı:", filtered_df.shape[0])


Kişi sayısı: 73


In [ ]:
data = data[missing_ratio <= 0.3]
data.head(10)

feature_name,subject_id,ALT_min,ALT_median,ALT_max,ALT_avg,AlkalinePhosphatase_min,AlkalinePhosphatase_median,AlkalinePhosphatase_max,AlkalinePhosphatase_avg,Magnesium_min,Magnesium_median,Magnesium_max,Magnesium_avg,Hemoglobin_min,Hemoglobin_median,Hemoglobin_max,Hemoglobin_avg,TotalCalcium_min,TotalCalcium_median,TotalCalcium_max,TotalCalcium_avg,Phosphate_min,Phosphate_median,Phosphate_max,Phosphate_avg,Temperature_min,Temperature_median,Temperature_max,Temperature_avg,Bicarbonate_min,Bicarbonate_median,Bicarbonate_max,Bicarbonate_avg,AnionGap_min,AnionGap_median,AnionGap_max,AnionGap_avg,PH_min,PH_median,PH_max,PH_avg,Chloride_min,Chloride_median,Chloride_max,Chloride_avg,Creatinine_min,Creatinine_median,Creatinine_max,Creatinine_avg,Glucose_min,Glucose_median,Glucose_max,Glucose_avg,Sodium_min,Sodium_median,Sodium_max,Sodium_avg,Potassium_min,Potassium_median,Potassium_max,Potassium_avg,Platelets_min,Platelets_median,Platelets_max,Platelets_avg,WBC_min,WBC_median,WBC_max,WBC_avg,Albumin_min,Albumin_median,Albumin_max,Albumin_avg,Neutrophils_min,Neutrophils_median,Neutrophils_max,Neutrophils_avg,OxygenSaturation_min,OxygenSaturation_median,OxygenSaturation_max,OxygenSaturation_avg,LactateDehydrogenase_min,LactateDehydrogenase_median,LactateDehydrogenase_max,LactateDehydrogenase_avg,AST_min,AST_median,AST_max,AST_avg,TotalBilirubin_min,TotalBilirubin_median,TotalBilirubin_max,TotalBilirubin_avg,ProthrombinTime_PT_min,ProthrombinTime_PT_median,ProthrombinTime_PT_max,ProthrombinTime_PT_avg,INR_min,INR_median,INR_max,INR_avg,BUN_min,BUN_median,BUN_max,BUN_avg,aPTT_min,aPTT_median,aPTT_max,aPTT_avg,MeanArterialPressure_min,MeanArterialPressure_median,MeanArterialPressure_max,MeanArterialPressure_avg,PaO2_min,PaO2_median,PaO2_max,PaO2_avg,PCO2_min,PCO2_median,PCO2_max,PCO2_avg,LacticAcid_min,LacticAcid_median,LacticAcid_max,LacticAcid_avg,HeartRate_min,HeartRate_median,HeartRate_max,HeartRate_avg,SystolicBP_min,SystolicBP_median,SystolicBP_max,SystolicBP_avg,DiastolicBP_min,DiastolicBP_median,DiastolicBP_max,DiastolicBP_avg,SedationScore_min,SedationScore_median,SedationScore_max,SedationScore_avg,SpO2_min,SpO2_median,SpO2_max,SpO2_avg
0,10017851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.50,1.50,1.50,1.50,10.60,10.60,10.60,10.60,7.30,7.30,7.30,7.30,4.60,4.60,4.60,4.60,33.70,35.30,36.90,35.30,22.00,22.00,22.00,22.00,18.00,18.00,18.00,18.00,7.14,7.25,7.26,7.22,105.00,105.00,105.00,105.00,1.70,1.70,1.70,1.70,127.00,127.00,127.00,127.00,140.00,140.00,140.00,140.00,4.50,4.50,4.50,4.50,492.00,492.00,492.00,492.00,20.40,20.40,20.40,20.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.00,63.00,98.00,63.00,3.40,4.10,4.60,4.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.40,50.40,50.40,50.40,5.50,5.50,5.50,5.50,52.00,52.00,52.00,52.00,42.70,42.70,42.70,42.70,-12.00,-11.00,-10.00,-11.00,18.00,45.00,181.00,81.33,51.00,54.00,72.00,59.00,3.40,4.10,4.60,4.03,105.00,126.00,148.00,126.58,80.00,101.00,135.00,100.15,38.00,56.00,85.00,58.39,8.00,24.00,35.00,22.82,76.00,96.00,100.00,93.70
1,10020944,15.00,23.50,32.00,23.50,105.00,109.00,113.00,109.00,1.30,1.60,1.80,1.57,10.00,10.60,11.20,10.60,7.60,8.20,8.30,8.03,1.20,1.50,4.80,2.50,NaN,NaN,NaN,NaN,24.00,25.00,27.00,25.33,8.00,8.50,9.00,8.50,7.13,7.28,7.44,7.28,103.00,110.00,111.00,108.00,1.20,1.20,1.40,1.27,34.00,64.00,124.00,74.00,132.00,140.00,141.00,137.67,4.30,4.55,4.80,4.55,151.00,158.50,166.00,158.50,9.30,11.00,12.70,11.00,3.30,3.30,3.30,3.30,76.20,79.10,82.00,79.10,NaN,NaN,NaN,NaN,1.10,2.10,3.10,2.10,21.00,69.50,118.00,69.50,0.40,0.40,0.40,0.40,12.60,13.05,13.50,13.05,1.20,1.20,1.20,1.20,31.00,31.00,34.00,32.00,32.60,33.30,34.00,33.30,57.00,77.00,113.00,79.28,30.00,80.00,101.00,76.00,38.00,53.00,84.00,56.00,1.10,2.10,3.10,2.10,53.00,56.00,78.00,58.31,101.00,113.00,113.00,109.00,59.00,62.00,69.00,63.33,6.00,16.00,28.00,17.32,93.00,97.00,100.00,96.83
3,10032381,9.00,9.00,9.00,9.00,65.00,65.50,66.00,65.50,1.20,1.85,2.50,1.85,8.10,8.70,9.30,8.70,7.10,7.70,8.30,7.70,3.50,3.90,4.30,3.90,NaN,NaN,NaN,NaN,23.00,24.50,26.00,24.50,

Sütun bazında da %30’dan fazla eksik değere sahip değişkenler olup olmadığı incelenmiştir. Ancak bu oranı aşan herhangi bir sütun bulunmadığı için sütunlar üzerinde ek bir işlem yapılmamıştır.

In [ ]:
missing_ratio_cols = data.isnull().mean(axis=0)

# %30'dan fazla eksik değeri olan sütunlar
data = data.loc[:, missing_ratio_cols <= 0.3]


In [ ]:
data.shape

(585, 125)

In [ ]:
data.isnull().sum()

,0
feature_name,
subject_id,0
ALT_min,122
ALT_median,122
ALT_max,122
ALT_avg,122
AlkalinePhosphatase_min,124
AlkalinePhosphatase_median,124
AlkalinePhosphatase_max,124
AlkalinePhosphatase_avg,124


Bu adımda, her hastaya ait demografik ve klinik özellikler (age, SOFA, SAPSII, comorbidity vb.) meta_df veri çerçevesi oluşturulmuştur.  
Ardından, bu özet bilgiler; daha önce hesaplanmış vital bulgu ve laboratuvar testlerine ait istatistiksel özetler (data) ile subject_id üzerinden birleştirilmiştir.

In [ ]:
meta_df = df9.drop(columns=['itemid', 'valuenum']).drop_duplicates(subset=['subject_id'])

final_df = meta_df.merge(data, on='subject_id', how='right')

In [ ]:
final_df = final_df.drop(columns=['charttime', 'intime'])

In [ ]:
final_df.shape

(585, 146)

**Final adımda da dataframe'i csv dosyası olarak kaydediyoruz.**

In [ ]:
from google.colab import files
final_df.to_csv('final_df_24.csv', index=False)
files.download('final_df_24.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>